<a href="https://colab.research.google.com/github/poseidon2022/Retreival-Augumented-Generation/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install --upgrade fsspec==2024.6.1
!pip install -qU \
  langchain==0.0.300 \
  datasets==2.14.6 \
  pinecone-client==2.2.4 \
  tiktoken==0.5.1
!pip install langchain_google_genai
!pip install pypdf==3.1.0

  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.6.1-py3-none-any.whl (177 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.10.0
    Uninstalling fsspec-2023.10.0:
      Successfully uninstalled fsspec-2023.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.14.6 requires fsspec[http]<=2023.10.0,>=2023.1.0, but you have fsspec 2024.6.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.10.0 which is incompatible.
langchain-core 0.2.38 requires langsmith<0.2.0,>=0.1.75, but you have langsmith 0.0.92 which is incompatible.
  Using cached langsmith-0.1.111-py3-none-any.wh

In [16]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KE')


In [17]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro",
                             google_api_key = GOOGLE_API_KEY)

#Example generation
llm.invoke("Write me a ballad about LangChain").content

"(Verse 1)\nIn a realm where data flows like a river wide,\nAnd models yearn for meaning, side by side,\nThere lived a language, potent and grand,\nLangChain, they called it, a helping hand.\n\n(Verse 2)\nWith chains of logic, it wove a spell,\nConnecting models, stories to tell,\nFrom GPT's wisdom to embeddings deep,\nLangChain awakened, secrets to keep.\n\n(Verse 3)\nIt spoke to databases, ancient and vast,\nUnveiling knowledge, forgotten in the past,\nWith APIs as bridges, it spanned the divide,\nBringing information, on its mighty tide.\n\n(Verse 4)\nIt whispered to agents, tasks to fulfill,\nAutomation's promise, with skillful thrill,\nFrom chatbots friendly to tools refined,\nLangChain empowered, leaving none behind.\n\n(Verse 5)\nBut with great power, a challenge arose,\nTo guide its potential, where wisdom flows,\nFor in the wrong hands, it could be misled,\nA tool for chaos, instead of being led.\n\n(Verse 6)\nSo let us wield LangChain, with hearts so true,\nTo build a future,

In [18]:
#let us have a continuous in-memory context remembering example
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Hi AI how are you today?"),
    ("ai", "I am great, how can I help you today?"),
    ("human", "I want to know more about string theory.")
]

In [19]:
res = llm.invoke(messages)
res.content

"That's a fascinating subject!  String theory is a complex and highly theoretical area of physics, but I can try to break it down into some key concepts. \n\nHere are some things to know about string theory:\n\n**The Basics**\n\n* **Everything is Made of Strings:**  String theory proposes that the fundamental building blocks of the universe are not point-like particles, but rather tiny, vibrating one-dimensional objects called strings. \n* **Vibrations Determine Properties:** Just like how different vibrations of a violin string produce different notes, different vibrations of these fundamental strings give rise to the different particles we observe (electrons, quarks, photons, etc.).\n* **Unifying Forces:** One of the main goals of string theory is to provide a unified description of all the fundamental forces of nature, including gravity, electromagnetism, and the strong and weak nuclear forces.\n\n**Key Concepts**\n\n* **Extra Dimensions:** String theory suggests that our universe m

In [20]:
messages.append(("ai", res.content))
messages

[('system', 'You are a helpful assistant.'),
 ('human', 'Hi AI how are you today?'),
 ('ai', 'I am great, how can I help you today?'),
 ('human', 'I want to know more about string theory.'),
 ('ai',
  "That's a fascinating subject!  String theory is a complex and highly theoretical area of physics, but I can try to break it down into some key concepts. \n\nHere are some things to know about string theory:\n\n**The Basics**\n\n* **Everything is Made of Strings:**  String theory proposes that the fundamental building blocks of the universe are not point-like particles, but rather tiny, vibrating one-dimensional objects called strings. \n* **Vibrations Determine Properties:** Just like how different vibrations of a violin string produce different notes, different vibrations of these fundamental strings give rise to the different particles we observe (electrons, quarks, photons, etc.).\n* **Unifying Forces:** One of the main goals of string theory is to provide a unified description of all

In [21]:
messages.append(("human", "tell me more"))
llm.invoke(messages).content

'Okay, let\'s dive a little deeper into some specific aspects of string theory:\n\n**1. Extra Dimensions: The Hidden Landscape**\n\n* Imagine a tightrope walker. From far away, they appear to move only in one dimension—along the rope. However, an ant crawling on the rope experiences an additional dimension: the circumference of the rope itself.\n* String theory suggests that our universe may have extra spatial dimensions curled up in a similar way, too small to be directly observed. These extra dimensions could be responsible for the properties of particles and forces we observe.\n* The way these extra dimensions are "compactified" (curled up) can lead to a vast number of possible universes with different physical laws, forming what\'s known as the "string theory landscape."\n\n**2. Unifying Forces: A Cosmic Symphony**\n\n* Each fundamental force in the universe is associated with a particular particle that carries that force. For example, photons carry the electromagnetic force.\n* St

Now we have our llm setup and the next thing is loading up a freash document. I setup a document on a fictional company named Kidus Melaku Simegne. But you can load up any document and test the functionality.

In [22]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key = GOOGLE_API_KEY)
#example embedding
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734449505806,
 0.01813092641532421]

In [33]:
#Now we have to load up our document and try to split it into smalled chunks that
#are later to be used for embedding
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("Kidus Melaku's company.pdf") #change this path to the file path you want
document = loader.load()



In [ ]:
#now we use a recursive text splitter to chunk up the text into a list of smaller texts
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=800,
   chunk_overlap=150
)

chunks = text_splitter.split_documents(document)
chunks
